In [13]:
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import preprocess, Preprocessor
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids=[1])

In [20]:
import numpy as np

from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
    ),
]

# Preprocess the data
preprocess(dataset, preprocessors, n_jobs=-1)

C:\Users\Joel\AppData\Roaming\Python\Python311\site-packages\braindecode\preprocessing\preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


In [22]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the window start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to
# define how windows should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


In [23]:
splitted = windows_dataset.split("session")
train_set = splitted['0train']  # Session train
test_set = splitted['1test']  # Session evaluation

In [24]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_channels = windows_dataset[0][0].shape[0]
input_window_samples = windows_dataset[0][0].shape[1]

model = ShallowFBCSPNet(
    n_channels,
    n_classes,
    n_times=input_window_samples,
    final_conv_length="auto",
    add_log_softmax=False,
)

# Display torchinfo table describing the model
print(model)


Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 22, 1125]             [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 22, 1125]             [1, 22, 1125, 1]          --                        --
├─Rearrange (dimshuffle): 1-2            [1, 22, 1125, 1]          [1, 1, 1125, 22]          --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1125, 22]          [1, 40, 1101, 1]          36,240                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1101, 1]          [1, 40, 1101, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1101, 1]          [1, 40, 1101, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1101, 1]          [1, 40, 69, 1]            --                        [75, 1]
├─Express

In [25]:
from skorch.callbacks import LRScheduler

from braindecode import EEGClassifier

lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 64
n_epochs = 10

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=None,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    classes=classes,
    max_epochs=n_epochs,
)
# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
clf.fit(train_set, y=None)

# evaluated the model after training
y_test = test_set.get_metadata().target
test_acc = clf.score(test_set, y=y_test)
print(f"Test acc: {(test_acc * 100):.2f}%")

  epoch    train_accuracy    train_loss      lr     dur
-------  ----------------  ------------  ------  ------
      1            0.3299       -0.4043  0.0006  1.7421
      2            0.3299       -2.1012  0.0006  1.5971
      3            0.3403       -3.7873  0.0006  2.0554
      4            0.3368       -5.3624  0.0005  2.1362
      5            0.3576       -7.2175  0.0004  1.6454
      6            0.3750       -8.4774  0.0003  1.8449
      7            0.3785       -9.0368  0.0002  1.8034
      8            0.4201       -9.6362  0.0001  1.9087
      9            0.4410      -10.1487  0.0000  1.7595
     10            0.4618      -10.1759  0.0000  1.9310
Test acc: 36.81%


In [26]:
predicted = clf.predict(test_set)
print(predicted)

[1 0 0 0 2 0 0 1 0 3 2 1 2 2 1 1 0 1 1 2 0 1 1 2 3 2 2 1 3 1 1 0 0 0 1 1 2
 0 1 0 0 0 1 0 0 1 3 1 1 3 2 2 1 2 0 2 1 0 2 2 2 1 2 0 2 0 3 1 2 2 0 0 1 1
 1 1 2 0 2 2 2 2 1 1 1 0 1 3 0 1 1 1 0 1 2 0 2 1 0 2 1 0 2 1 1 0 1 0 3 2 0
 2 0 2 1 1 2 1 1 1 2 0 0 2 2 2 1 0 2 1 0 2 1 2 1 2 1 1 1 2 0 0 2 1 2 2 1 0
 0 0 1 1 0 2 0 0 1 1 1 0 1 3 1 0 0 1 2 0 0 2 1 1 2 2 2 1 0 1 1 2 1 2 0 0 3
 3 0 2 0 1 2 0 1 2 1 0 1 2 1 1 1 0 1 2 2 1 1 1 0 0 1 1 0 0 1 0 1 1 2 1 1 0
 1 3 2 1 0 1 1 0 0 1 1 2 0 1 1 1 0 0 1 1 3 0 1 2 1 2 0 2 3 1 1 2 1 0 2 1 1
 1 2 0 0 0 0 1 2 0 1 2 3 1 0 1 1 2 1 0 2 0 1 0 0 1 0 2 2 0]


In [ ]:
print(test_set.get_metadata().target)